In [ ]:
!pip install streamlit --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 123.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.4 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok --quiet # Install the pyngrok module
from pyngrok import ngrok

In [ ]:
!ngrok config add-authtoken xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
...............................

In [ ]:
!kill -9 $(lsof -t -i:8000) 2>/dev/null || echo "No existing process"


No existing process


In [ ]:
!pip install --quiet flask transformers torch networkx matplotlib scikit-learn safetensors pyvis soundfile psutil GPUtil
# no dynamic quant error on GPU for int8 => fallback to CPU

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.7 MB/s eta 0:00:00


In [ ]:
!pkill -f ngrok

In [1]:
import os
import time
import random
import threading
import requests
import torch
import torch.nn as nn
import networkx as nx
from flask import Flask, request, jsonify, Response
from collections import deque
from subprocess import Popen, PIPE, STDOUT
import traceback
import pandas as pd

# ---------------- USER CONFIG ----------------
SCENARIO = "ngrok"  # "domain", "ngrok", or "cloudflared"
HUGGINGFACE_TOKEN = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"        # <-- Replace
NGROK_AUTH_TOKEN = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx" # <-- Replace
PORT = 8000

# ---------------- LOG BUFFER ----------------
LOG_BUFFER = deque(maxlen=100)
def log_msg(msg):
    LOG_BUFFER.append(msg)
    print(msg, flush=True)

# -------------- KILL ANYTHING ON PORT=8000 --------------
def kill_port_8000():
    try:
        os.system("pkill -f ngrok 2>/dev/null || echo 'No old ngrok.'")
        os.system("pkill -f cloudflared 2>/dev/null || echo 'No old cloudflared.'")
        os.system("kill -9 $(lsof -t -i:8000) 2>/dev/null || echo 'No old process on 8000.'")
    except:
        pass

log_msg(f"🔴 Killing anything on port={PORT} + old ngrok/cloudflared.")
kill_port_8000()

# --------------- INSTALL DEPS ---------------
log_msg("📦 Installing BFS dependencies quietly...")
deps = [
    "flask", "transformers", "torch", "accelerate", "peft", "bitsandbytes", "pyngrok",
    "flask_cloudflared", "networkx", "matplotlib", "pyvis", "psutil", "requests",
    "scipy", "GPUtil", "plotly", "huggingface_hub", "pandas"
]
os.system("pip install --quiet " + " ".join(deps))

# --------------- HUGGING FACE LOGIN ---------------
log_msg("🔑 Logging into Hugging Face...")
try:
    from huggingface_hub import login
    login(HUGGINGFACE_TOKEN)
except Exception as e:
    log_msg(f"❗ Could not login => {e}")

# --------------- FLASK APP ---------------
app = Flask(__name__, static_folder="static")
if not os.path.exists("static"):
    os.makedirs("static")

LOG_DIR = "model_logs"
os.makedirs(LOG_DIR, exist_ok=True)

# --------------- BFS GLOBALS ---------------
RL_CHOICE = "No RL"
HYP_GRAPH = None
COMP_GRAPH = None
OLD_HYP_GRAPH = None
OLD_COMP_GRAPH = None
MODEL_CACHE = {}
Q_TABLE = {s: [random.uniform(-1,1) for _ in range(4)] for s in range(10)}

GPU_MEMORY_SCALE = {
    "T4": 1.0,
    "A100": 0.5,
    "V100": 0.7,
    "TPU": 0.4
}

BFS_CANCEL = False
BFS_RUNNING = False
BFS_FAIL = False
BFS_THREAD = None

CURRENT_MODEL_LABEL = None
CURRENT_QUANT = None
CURRENT_REASONING = None
CURRENT_RL = None

FLOP_RANDOM_FACTORS = {}

# --------------- MODEL LIST ---------------
AVAILABLE_MODELS = {
    "BERT (base)": "bert-base-uncased",
    "GPT-2": "gpt2",
    "DistilBERT": "distilbert-base-uncased",
    "Falcon-7B": "tiiuae/falcon-7b-instruct",
    "Flan-UL2 (20B)": "google/flan-ul2",
    "Qwen-30B": "Qwen/Qwen-30B"
}
MODEL_SIZE_INFO = {
    "BERT (base)":110e6,
    "GPT-2":124e6,
    "DistilBERT":66e6,
    "Falcon-7B":7e9,
    "Flan-UL2 (20B)":20e9,
    "Qwen-30B":30e9
}
QUANT_METHODS = [
    "None (FP32)","FP16","INT8","4-bit (bitsandbytes)","8-bit (bitsandbytes)"
]
REASONING_METHODS = [
    "No reasoning","Monte Carlo Tree Search (MCTS)",
    "Chain of Thought (CoT)","Meta-CoT","Neural-Symbolic"
]
RL_ALGORITHMS = [
    "No RL","Q-Learning","Deep Q-Network (DQN)",
    "Policy Gradient","Actor-Critic","PPO"
]
ATTN_COMPLEXITY_OPTS = [
    "Quadratic (n^2)","Logarithmic (n log n)","Linear (n)"
]

FIXED_LIN_COMP = "Linear (n)"
FIXED_DEF_COMP = "Linear (n)"

# =============== BFS UTILS ===============
def keep_largest_connected_component(nx_graph):
    if nx_graph.number_of_nodes()==0:
        return nx_graph
    ug= nx_graph.to_undirected()
    comps= list(nx.connected_components(ug))
    if not comps:
        return nx_graph
    largest= max(comps, key=len)
    return nx_graph.subgraph(largest).copy()

def compute_bfs_levels(nx_graph):
    if nx_graph.number_of_nodes()==0:
        return
    for n in nx_graph.nodes:
        nx_graph.nodes[n]["BFS_distance"]=0
    root_candidates= [n for n in nx_graph.nodes if "input" in n.lower()]
    if not root_candidates:
        in_degs= dict(nx_graph.in_degree())
        zero_in= [nd for nd,deg in in_degs.items() if deg==0]
        if zero_in:
            root_candidates= zero_in
        if not root_candidates:
            root_candidates= [list(nx_graph.nodes)[0]]
    visited= set()
    queue= []
    for r in root_candidates:
        queue.append((r,0))
        nx_graph.nodes[r]["BFS_distance"]=0
        visited.add(r)
    while queue:
        if BFS_CANCEL:
            log_msg("🔴 BFS CANCEL => early exit BFS levels.")
            return
        current, dist= queue.pop(0)
        for neighbor in nx_graph.neighbors(current):
            if neighbor not in visited:
                visited.add(neighbor)
                nx_graph.nodes[neighbor]["BFS_distance"]= dist+1
                queue.append((neighbor, dist+1))

def stable_flop_factor(node_str):
    if node_str not in FLOP_RANDOM_FACTORS:
        FLOP_RANDOM_FACTORS[node_str]= random.uniform(0.5,2.0)
    return FLOP_RANDOM_FACTORS[node_str]

def interpret_time_cost(node_str, attn_comp, lin_comp, def_comp, seq_len):
    low= node_str.lower()
    if "attention" in low or "matmul" in low:
        chosen= attn_comp
    elif "linear" in low:
        chosen= lin_comp
    else:
        chosen= def_comp
    import math
    if chosen.startswith("Quadratic"):
        base= seq_len**2
    elif chosen.startswith("Logarithmic"):
        base= int(seq_len*math.log2(seq_len+1))
    else:
        base= seq_len
    factor= stable_flop_factor(node_str)
    return base* factor

def compute_node_flops(node_str, attn_comp, lin_comp, def_comp, seq_len):
    base_t= interpret_time_cost(node_str, attn_comp, lin_comp, def_comp, seq_len)
    return base_t*5.0

# ------------- HOOK-BASED (ONLY) -------------
def hook_based_extract(model, tokenizer):
    import torch
    import networkx as nx
    g= nx.DiGraph()
    call_seq=[]
    def make_hook(mod_name):
        def fwd_hook(module,inp,outp):
            if BFS_CANCEL:
                return
            call_seq.append(mod_name)
        return fwd_hook
    for nm, subm in model.named_modules():
        subm.register_forward_hook(make_hook(nm))
    device= next(model.parameters()).device
    enc= tokenizer("Hello hooking approach!", return_tensors="pt").to(device)
    with torch.no_grad():
        _= model(**enc)
    # build hooking graph
    for nm in set(call_seq):
        g.add_node("COMP_hook_"+ nm, BFS_distance=0,mem=0,timecost=0,target_mem=0,target_time=0,flops=0)
    for i in range(len(call_seq)-1):
        parent= "COMP_hook_"+ call_seq[i]
        child=  "COMP_hook_"+ call_seq[i+1]
        g.add_edge(parent, child)
    node_count= g.number_of_nodes()
    edge_count= g.number_of_edges()
    log_msg(f"Hook-based => discovered {node_count} nodes, {edge_count} edges from forward pass.")
    return g

def fallback_comp_graph_for_model(model):
    import networkx as nx
    g= nx.DiGraph()
    for name,_ in model.named_modules():
        node_name= "COMP_"+ name
        g.add_node(node_name,BFS_distance=0,mem=0,timecost=0,target_mem=0,target_time=0,flops=0)
        if "." in name:
            parent= "COMP_"+ name.rsplit(".",1)[0]
            child= "COMP_"+ name
            g.add_edge(parent, child)
    return g

def extract_comp_graph(model, tokenizer):
    # Just hooking => if hooking <2 nodes => fallback
    try:
        g_hook= hook_based_extract(model, tokenizer)
        if g_hook.number_of_nodes()>=2:
            log_msg("✅ Hook-based => returning hooking graph.")
            return g_hook
        else:
            log_msg("⚠️ Hook-based => <2 nodes => fallback.")
    except Exception as e:
        log_msg(f"Hook => fail => {e}")
    log_msg("⚠️ named_modules fallback => minimal graph.")
    return fallback_comp_graph_for_model(model)

def extract_hypergraph(model):
    import networkx as nx
    g= nx.DiGraph()
    for name,_ in model.named_modules():
        node_name= "HYP_"+ name
        g.add_node(node_name)
        if "." in name:
            parent= "HYP_"+ name.rsplit(".",1)[0]
            child=  "HYP_"+ name
            g.add_edge(parent, child)
    return g

# --------------- BFS => color => concurrency ---------------
def mark_graph_changes(new_graph, old_graph):
    if not old_graph:
        for n in new_graph.nodes:
            new_graph.nodes[n]["color"]="green"
        for e in new_graph.edges:
            new_graph[e[0]][e[1]]["color"]="green"
        return
    old_nodes= old_graph.nodes(data=True)
    old_edges= set(old_graph.edges())
    for n,d in new_graph.nodes(data=True):
        if n not in old_graph:
            d["color"]="green"
        else:
            old_data= old_nodes[n]
            changed=False
            fields=["BFS_distance","mem","timecost","target_mem","target_time","flops"]
            for f in fields:
                old_val= old_data.get(f,None)
                new_val= d.get(f,None)
                if old_val!= new_val:
                    changed=True
                    break
            if changed:
                d["color"]="red"
            else:
                d["color"]="blue"
    new_edges= set(new_graph.edges())
    for e in new_edges:
        if e not in old_edges and (e[1], e[0]) not in old_edges:
            new_graph[e[0]][e[1]]["color"]="green"
        else:
            new_graph[e[0]][e[1]]["color"]="gray"

def update_q_table(state,action,reward):
    old_val= Q_TABLE[state][action]
    new_val= old_val+0.1*(reward- old_val)
    Q_TABLE[state][action]= new_val

# --------------- auto_force_quant + load_model ---------------
def auto_force_quant(model_label, user_choice):
    size= MODEL_SIZE_INFO.get(model_label,0)
    if size>=1e9 and size<10e9:
        return "8-bit (bitsandbytes)"
    elif size>=10e9:
        return "4-bit (bitsandbytes)"
    else:
        return user_choice

def load_model(hf_id, model_label, quant="None (FP32)"):
    import torch.nn as nn
    import torch
    forced_quant= auto_force_quant(model_label, quant)
    if forced_quant!= quant:
        log_msg(f"⚠️ Overriding user quant={quant} => forced to {forced_quant}.")
        quant= forced_quant
    device= "cuda" if torch.cuda.is_available() else "cpu"
    if quant=="INT8" and device=="cuda":
        device="cpu"
    cache_key= f"{hf_id}_{quant}_{device}"
    if cache_key in MODEL_CACHE:
        return MODEL_CACHE[cache_key]["model"], MODEL_CACHE[cache_key]["tokenizer"]

    from transformers import AutoModel, AutoModelForCausalLM, AutoTokenizer

    def try_causallm():
        return AutoModelForCausalLM.from_pretrained(hf_id)
    def try_auto():
        return AutoModel.from_pretrained(hf_id)

    try:
        model= try_causallm()
    except:
        model= try_auto()

    tokenizer= AutoTokenizer.from_pretrained(hf_id)
    if not tokenizer.pad_token_id:
        tokenizer.pad_token_id= tokenizer.eos_token_id or 0

    if quant=="FP16":
        model= model.half()
    elif quant=="INT8":
        model= torch.quantization.quantize_dynamic(model,{nn.Linear},dtype=torch.qint8)
    elif quant in ["4-bit (bitsandbytes)","8-bit (bitsandbytes)"]:
        from transformers import BitsAndBytesConfig,AutoModelForCausalLM
        if "4-bit" in quant:
            bnb_config= BitsAndBytesConfig(load_in_4bit=True,bnb_4bit_compute_dtype=torch.float16)
        else:
            bnb_config= BitsAndBytesConfig(load_in_8bit=True)
        model= AutoModelForCausalLM.from_pretrained(hf_id, quantization_config=bnb_config, device_map="auto")
        model.eval()
        MODEL_CACHE[cache_key]={"model":model,"tokenizer":tokenizer}
        return model, tokenizer

    model.eval()
    if quant not in ["INT8","4-bit (bitsandbytes)","8-bit (bitsandbytes)"]:
        model.to(device)
    MODEL_CACHE[cache_key]={"model":model,"tokenizer":tokenizer}
    return model, tokenizer

# -------------- BFS => background --------------
def run_bfs_in_background(model, tokenizer, model_label, quant, seq_len, attn_comp, lin_comp, def_comp):
    global BFS_CANCEL, BFS_RUNNING, BFS_FAIL
    global HYP_GRAPH, COMP_GRAPH, OLD_HYP_GRAPH, OLD_COMP_GRAPH
    BFS_CANCEL= False
    BFS_RUNNING= True
    BFS_FAIL= False
    log_msg("🔵 BFS background thread => start")
    try:
        new_hyp= extract_hypergraph(model)
        new_hyp= keep_largest_connected_component(new_hyp)
        if BFS_CANCEL:
            BFS_RUNNING=False
            log_msg("🔴 BFS CANCEL => early exit (hyper).")
            return

        new_comp= extract_comp_graph(model, tokenizer)
        new_comp= keep_largest_connected_component(new_comp)
        if BFS_CANCEL:
            BFS_RUNNING=False
            log_msg("🔴 BFS CANCEL => early exit (comp).")
            return

        compute_bfs_levels(new_comp)
        if BFS_CANCEL:
            BFS_RUNNING=False
            log_msg("🔴 BFS CANCEL => early exit (bfs_levels).")
            return

        mark_graph_changes(new_hyp, OLD_HYP_GRAPH)
        mark_graph_changes(new_comp, OLD_COMP_GRAPH)

        # highlight outdegree>1 => orange
        out_degs= new_comp.out_degree()
        for nd, deg in out_degs:
            if deg>1:
                new_comp.nodes[nd]["color"]="orange"

        from copy import deepcopy
        HYP_GRAPH= new_hyp
        COMP_GRAPH= new_comp
        OLD_HYP_GRAPH= deepcopy(new_hyp)
        OLD_COMP_GRAPH= deepcopy(new_comp)

        scale_factor= GPU_MEMORY_SCALE.get("T4",1.0)
        new_comp.graph["seq_len"]= seq_len
        new_comp.graph["attn_comp"]= attn_comp
        new_comp.graph["lin_comp"]= lin_comp
        new_comp.graph["def_comp"]= def_comp
        new_comp.graph["scale_factor"]= scale_factor

        for n,d2 in new_comp.nodes(data=True):
            if BFS_CANCEL:
                BFS_RUNNING=False
                log_msg("🔴 BFS CANCEL => early exit (fill mem/time).")
                return
            BFS_d= d2.get("BFS_distance",0)
            fallback_elems= random.randint(1,10)*1000
            shape_mem= (fallback_elems*4)/1024
            shape_mem*= random.uniform(0.8,1.2)
            base_t= interpret_time_cost(n, attn_comp, lin_comp, def_comp, seq_len)
            base_f= compute_node_flops(n, attn_comp, lin_comp, def_comp, seq_len)
            mem_init= (BFS_d+1)* shape_mem* scale_factor
            time_init= (BFS_d+1)* base_t
            flop_val= (BFS_d+1)* base_f

            d2["mem"]= mem_init
            d2["target_mem"]= mem_init
            d2["timecost"]= time_init
            d2["target_time"]= time_init*1.5
            d2["flops"]= flop_val

        log_msg("🔵 BFS background thread => complete")
    except Exception as e:
        BFS_FAIL= True
        err_msg= f"❌ BFS exception => {e}\n{traceback.format_exc()}"
        log_msg(err_msg)
    BFS_RUNNING=False

# -------------- MAIN UI + Auto-Reload --------------
@app.route("/", methods=["GET","POST"])
def main_page():
    global BFS_CANCEL, BFS_RUNNING, BFS_FAIL, BFS_THREAD
    global RL_CHOICE, CURRENT_MODEL_LABEL, CURRENT_QUANT, CURRENT_REASONING, CURRENT_RL
    global COMP_GRAPH,HYP_GRAPH

    message_html=""
    # A small JS script to auto-reload every 2s if BFS_RUNNING
    auto_refresh_js = ""
    if BFS_RUNNING:
        # auto-refresh every 2 seconds
        auto_refresh_js = """
        <script>
        setTimeout(() => { window.location.reload(); }, 2000);
        </script>
        """

    if request.method=="POST":
        act= request.form.get("act","")
        if act=="cancel_bfs":
            if BFS_RUNNING:
                BFS_CANCEL= True
                message_html= "<p style='color:red;'>Requested BFS cancel. Checking background thread...</p>"
            else:
                message_html= "<p>No BFS is currently running to cancel.</p>"
        else:
            # user started BFS
            model_label= request.form.get("model","BERT (base)")
            hf_id= AVAILABLE_MODELS.get(model_label,"bert-base-uncased")
            quant= request.form.get("quant","None (FP32)")
            reasoning= request.form.get("reason","No reasoning")
            rl_algo= request.form.get("rl_algo","No RL")
            RL_CHOICE= rl_algo
            CURRENT_MODEL_LABEL= model_label
            CURRENT_QUANT= quant
            CURRENT_REASONING= reasoning
            CURRENT_RL= rl_algo

            seq_len_str= request.form.get("seq_len","32").strip()
            attn_comp_choice= request.form.get("attn_comp","Quadratic (n^2)")
            try:
                seq_len= int(seq_len_str)
                if seq_len<1: seq_len=1
            except:
                seq_len=32

            try:
                BFS_CANCEL=False
                BFS_RUNNING=True
                BFS_FAIL=False
                model, tokenizer= load_model(hf_id, model_label, quant)
                if rl_algo!="No RL":
                    s= random.randint(0,9)
                    a= random.randint(0,3)
                    r= random.uniform(-1,1)
                    update_q_table(s,a,r)

                BFS_THREAD= threading.Thread(
                    target=run_bfs_in_background,
                    daemon=True,
                    args=(model, tokenizer, model_label, quant, seq_len, attn_comp_choice, FIXED_LIN_COMP, FIXED_DEF_COMP)
                )
                BFS_THREAD.start()
                message_html= f"<p><b>Spawned BFS thread => {model_label}, quant={quant}, RL={rl_algo}.</b></p>"
            except Exception as e:
                BFS_RUNNING=False
                BFS_FAIL=True
                message_html= f"<p style='color:red;'>Model/BFS init failed => {e}</p>"

    if BFS_RUNNING:
        message_html+= "<p style='color:green;'>BFS is running in background. You can cancel below. (Auto-refresh ON)</p>"
    else:
        message_html+= "<p>BFS is not currently running.</p>"
    if BFS_FAIL:
        message_html+= "<p style='color:red;'>BFS FAILED => see debug logs for details. No graphs displayed!</p>"

    model_info_html=""
    if CURRENT_MODEL_LABEL or CURRENT_QUANT or CURRENT_RL:
        model_info_html= f"""
        <p style="color:blue;">
          <b>Currently analyzing:</b>
          Model={CURRENT_MODEL_LABEL or '-'},
          Quant={CURRENT_QUANT or '-'},
          Reasoning={CURRENT_REASONING or '-'},
          RL={CURRENT_RL or '-'}
        </p>
        """

    model_opts= "\n".join([f'<option>{m}</option>' for m in AVAILABLE_MODELS])
    quant_opts= "\n".join([f'<option>{q}</option>' for q in QUANT_METHODS])
    reasoning_opts= "\n".join([f'<option>{r}</option>' for r in REASONING_METHODS])
    rl_opts= "\n".join([f'<option>{ra}</option>' for ra in RL_ALGORITHMS])
    attn_opts= "\n".join([f'<option>{ac}</option>' for ac in ATTN_COMPLEXITY_OPTS])

    if BFS_RUNNING or BFS_FAIL or (not COMP_GRAPH):
        graph_html= "<p>No BFS graph => run BFS or wait for BFS completion (or BFS failed/tracing error).</p>"
        qspace_html= ""
        metrics_html= ""
    else:
        graph_html= show_side_by_side_graphs()
        if CURRENT_RL!="No RL":
            qspace_html= build_qspace_iframe()
        else:
            qspace_html=""
        if COMP_GRAPH.number_of_nodes()>0:
            metrics_html= build_metrics_iframe()
        else:
            metrics_html=""

    html= f"""
    <html>
    <head>
      <title>AI Interpretability Tool</title>
      {auto_refresh_js}
    </head>
    <body style="background-color:#E5D3F2;text-align:center;">
      <h1>AI Interpretability Tool</h1>
      <p><a href="/debug_logs" target="_blank">View debug logs</a></p>

      <form method="POST">
        <h3>Start BFS Analysis</h3>
        Model:
        <select name="model">{model_opts}</select>
        Quant:
        <select name="quant">{quant_opts}</select>
        Reasoning:
        <select name="reason">{reasoning_opts}</select>
        RL Algo:
        <select name="rl_algo">{rl_opts}</select>
        <br><br>
        Sequence length: <input name="seq_len" value="32" size=4>
        <br>
        Attn complexity:
        <select name="attn_comp">{attn_opts}</select>
        <br><br>
        <input type="hidden" name="act" value="run"/>
        <input type="submit" value="Run BFS"/>
      </form>

      <form method="POST" style="margin-top:20px;">
        <input type="hidden" name="act" value="cancel_bfs"/>
        <input type="submit" value="Cancel BFS"/>
      </form>

      <hr>
      {message_html}
      {model_info_html}
      <hr>
      {graph_html}
      {qspace_html}
      {metrics_html}

      <hr>
      <p>
        <a href="/bottleneck_analyze">Bottleneck Analysis</a> |
        <a href="/plot_distributions" target="_blank">Distribution Plots</a> |
        <a href="/optimize_model" target="_blank">Optimize Model</a>
      </p>
    </body></html>
    """
    return html

# -------------- BOTTLENECK --------------
@app.route("/bottleneck_analyze", methods=["GET","POST"])
def bottleneck_analyze():
    global COMP_GRAPH
    if not COMP_GRAPH:
        return "<p>No BFS => run BFS first.</p>"
    if request.method=="POST":
        std_threshold_str= request.form.get("std_threshold","2").strip()
        opt_choice= request.form.get("opt_choice","pruning")
        try:
            std_threshold= float(std_threshold_str)
        except:
            std_threshold=2.0

        time_vals= [d["timecost"] for _, d in COMP_GRAPH.nodes(data=True)]
        mem_vals=  [d["mem"] for _, d in COMP_GRAPH.nodes(data=True)]
        flop_vals= [d["flops"] for _, d in COMP_GRAPH.nodes(data=True)]
        def mean_std(vals):
            if not vals: return (0,0)
            m= sum(vals)/len(vals)
            var= sum((v-m)**2 for v in vals)/ len(vals)
            return (m, var**0.5)

        t_mean,t_std= mean_std(time_vals)
        m_mean,m_std= mean_std(mem_vals)
        f_mean,f_std= mean_std(flop_vals)

        bottleneck_nodes=[]
        for n,d in COMP_GRAPH.nodes(data=True):
            tv= d["timecost"]
            mv= d["mem"]
            fv= d["flops"]
            time_exceed= (tv> t_mean+ std_threshold*t_std) if t_std>0 else False
            mem_exceed= (mv> m_mean+ std_threshold*m_std) if m_std>0 else False
            flop_exceed= (fv> f_mean+ std_threshold*f_std) if f_std>0 else False
            if time_exceed or mem_exceed or flop_exceed:
                d["color"]="magenta"
                bottleneck_nodes.append(n)

        if opt_choice.lower()!="none":
            for bn in bottleneck_nodes:
                nd= COMP_GRAPH.nodes[bn]
                nd["timecost"]*=0.7
                nd["mem"]*=0.7
                nd["flops"]*=0.7

        return f"<h3>Bottleneck Analysis</h3><p>Bottleneck nodes => {bottleneck_nodes}, threshold={std_threshold}, method={opt_choice}</p><p><a href='/'>Back</a></p>"

    return """
    <h2>Bottleneck Analysis</h2>
    <form method="POST">
      Stdev threshold: <input name="std_threshold" value="2"/><br><br>
      Optimization method:
      <select name="opt_choice">
        <option>pruning</option>
        <option>quantization</option>
        <option>tensor-fusion</option>
        <option>none</option>
      </select><br><br>
      <input type='submit' value='Analyze'/>
    </form>
    <p><a href='/'>Back</a></p>
    """

# -------------- DISTRIBUTIONS --------------
@app.route("/plot_distributions")
def plot_distributions():
    global COMP_GRAPH
    if not COMP_GRAPH or COMP_GRAPH.number_of_nodes()==0:
        return "<p>No BFS => run BFS first.</p>"
    time_vals= [d["timecost"] for _,d in COMP_GRAPH.nodes(data=True)]
    mem_vals=  [d["mem"] for _,d in COMP_GRAPH.nodes(data=True)]
    flop_vals= [d["flops"] for _,d in COMP_GRAPH.nodes(data=True)]
    import plotly.subplots as sp
    import plotly.graph_objs as go
    fig= sp.make_subplots(rows=1,cols=3, subplot_titles=("Time Dist","Memory Dist","FLOPs Dist"))
    fig.add_trace(go.Histogram(x=time_vals), row=1,col=1)
    fig.add_trace(go.Histogram(x=mem_vals),  row=1,col=2)
    fig.add_trace(go.Histogram(x=flop_vals), row=1,col=3)
    fig.update_layout(title_text="Distribution of Time/Memory/FLOPs", showlegend=False)
    dist_fname="static/dist_plots.html"
    fig.write_html(dist_fname, include_plotlyjs='cdn')
    return f"""
    <h2>Distribution Plots</h2>
    <iframe src="/{dist_fname}" style="width:100%; height:600px; border:none;"></iframe>
    <p><a href="/">Back</a></p>
    """

# -------------- EXPORT BFS / Q --------------
@app.route("/export_data")
def export_data():
    global COMP_GRAPH,HYP_GRAPH
    if not COMP_GRAPH:
        return "No BFS => run BFS first."
    graph_choice= request.args.get("graph","comp").lower()
    fmt= request.args.get("fmt","csv").lower()
    nxg= HYP_GRAPH if graph_choice=="hyp" else COMP_GRAPH
    rows=[]
    for n,d in nxg.nodes(data=True):
        row={
            "node_id":n,
            "BFS_distance":d.get("BFS_distance",0),
            "mem":d.get("mem",0),
            "timecost":d.get("timecost",0),
            "flops":d.get("flops",0),
            "color":d.get("color",""),
        }
        rows.append(row)
    if fmt=="json":
        return jsonify(rows)
    else:
        df= pd.DataFrame(rows)
        csv_data= df.to_csv(index=False)
        return Response(
            csv_data,
            mimetype="text/csv",
            headers={"Content-disposition":"attachment; filename=bfs_data.csv"}
        )

@app.route("/export_qspace")
def export_qspace():
    rows=[]
    for s in range(10):
        for a in range(4):
            val= Q_TABLE[s][a]
            rows.append({"state":s,"action":a,"q_value":val})
    fmt= request.args.get("fmt","csv").lower()
    if fmt=="json":
        return jsonify(rows)
    else:
        import pandas as pd
        df= pd.DataFrame(rows)
        csv_data= df.to_csv(index=False)
        return Response(
            csv_data,
            mimetype="text/csv",
            headers={"Content-disposition":"attachment; filename=qspace_data.csv"}
        )

# -------------- Show side-by-side BFS => hyper / comp --------------
from pyvis.network import Network
import plotly.graph_objs as go

def show_side_by_side_graphs():
    global HYP_GRAPH, COMP_GRAPH, BFS_FAIL
    if BFS_FAIL:
        return "<p style='color:red;'>BFS failed => no graphs to show.</p>"
    if not HYP_GRAPH or not COMP_GRAPH:
        return "<p>No BFS graphs => run BFS or BFS not done or BFS failed/tracing error.</p>"

    net_h= Network(height="600px",width="100%",directed=False)
    net_h.barnes_hut()
    net_h.set_options("""
    var options={
      "physics": {
        "enabled":true,
        "stabilization":{"iterations":500}
      }
    }""")
    for n,d in HYP_GRAPH.nodes(data=True):
        clr= d.get("color","blue")
        net_h.add_node(n,label=n,color=clr)
    for u,v in HYP_GRAPH.edges():
        net_h.add_edge(u,v,color="gray")
    hyper_fname="static/hypergraph_full.html"
    net_h.write_html(hyper_fname, notebook=False, open_browser=False)

    net_c= Network(height="600px",width="100%",directed=False)
    net_c.barnes_hut()
    net_c.set_options("""
    var options={
      "physics": {
        "enabled":true,
        "stabilization":{"iterations":500}
      }
    }""")
    for n,d in COMP_GRAPH.nodes(data=True):
        clr= d.get("color","blue")
        BFS_d= d.get("BFS_distance",0)
        memv= d.get("mem",0)
        timv= d.get("timecost",0)
        flps= d.get("flops",0)
        title_txt= f"BFS={BFS_d}, mem={memv:.2f}, time={timv:.2f}, flops={flps}"
        net_c.add_node(n,label=n,color=clr,title=title_txt)
    for u,v in COMP_GRAPH.edges():
        clr= COMP_GRAPH[u][v].get("color","gray")
        net_c.add_edge(u,v,color=clr)
    comp_fname="static/compgraph_full.html"
    net_c.write_html(comp_fname, notebook=False, open_browser=False)

    return f"""
    <div style="display:flex; flex-direction:row; gap:20px; justify-content:space-evenly; margin-top:10px;">
      <div style="flex:1;">
        <h3 style="text-align:center; font-size:2em;">Hypergraph</h3>
        <iframe src="/{hyper_fname}" style="width:100%; height:600px; border:none;"></iframe>
        <p style="text-align:center;">
          <a href="/export_data?graph=hyp&fmt=csv" target="_blank">CSV</a> |
          <a href="/export_data?graph=hyp&fmt=json" target="_blank">JSON</a>
        </p>
      </div>
      <div style="flex:1;">
        <h3 style="text-align:center; font-size:2em;">Computational Graph</h3>
        <iframe src="/{comp_fname}" style="width:100%; height:600px; border:none;"></iframe>
        <p style="text-align:center;">
          <a href="/export_data?graph=comp&fmt=csv" target="_blank">CSV</a> |
          <a href="/export_data?graph=comp&fmt=json" target="_blank">JSON</a>
        </p>
      </div>
    </div>
    """

def build_qspace_iframe():
    xs,ys,zs=[],[],[]
    for s in range(10):
        for a in range(4):
            xs.append(s)
            ys.append(a)
            zs.append(Q_TABLE[s][a])
    fig= go.Figure(data=[go.Scatter3d(
        x=xs,y=ys,z=zs,
        mode='markers',
        marker=dict(size=5,color=zs,colorscale='Viridis'),
        text=[f"Q({s},{a})={val:.2f}" for s,a,val in zip(xs,ys,zs)]
    )])
    fig.update_layout(
        title={'text':"Q(s,a) 3D space",'x':0.5,'xanchor':'center','font':{'size':24}},
        scene=dict(
            xaxis_title='State(s)',
            yaxis_title='Action(a)',
            zaxis_title='Q-value'
        )
    )
    qspace_fname="static/qspace.html"
    fig.write_html(qspace_fname, include_plotlyjs='cdn')
    return f"""
    <div style="text-align:center; margin-top:20px;">
      <h3 style="font-size:2em;">Q-space 3D</h3>
      <iframe src="/{qspace_fname}" style="width:100%; height:600px; border:none;"></iframe>
      <p>
        <a href="/export_qspace?fmt=csv" target="_blank">CSV</a> |
        <a href="/export_qspace?fmt=json" target="_blank">JSON</a>
      </p>
    </div>
    """

def build_metrics_iframe():
    global COMP_GRAPH
    if not COMP_GRAPH or COMP_GRAPH.number_of_nodes()==0:
        return ""
    import plotly.graph_objs as go
    mem_vals=[]
    time_vals=[]
    flop_vals=[]
    labels=[]
    for n,d in COMP_GRAPH.nodes(data=True):
        mem_vals.append(d["mem"])
        time_vals.append(d["timecost"])
        flop_vals.append(d["flops"])
        labels.append(n)
    fig= go.Figure(data=[go.Scatter3d(
        x=mem_vals,
        y=time_vals,
        z=flop_vals,
        mode='markers',
        marker=dict(size=5,color=time_vals,colorscale='Viridis'),
        text=labels
    )])
    fig.update_layout(
        title={'text':"Memory/Time/FLOPs (3D)","x":0.5,"xanchor":"center","font":{"size":24}},
        scene=dict(
            xaxis_title='Memory(KB)',
            yaxis_title='Time(ms)',
            zaxis_title='FLOPs'
        )
    )
    metric_fname="static/metrics_3d.html"
    fig.write_html(metric_fname, include_plotlyjs='cdn')
    return f"""
    <div style="text-align:center; margin-top:20px;">
      <h3 style="font-size:2em;">Memory/Time/FLOPs 3D</h3>
      <iframe src="/{metric_fname}" style="width:100%; height:600px; border:none;"></iframe>
      <p><a href="/export_data?graph=comp&fmt=csv" target="_blank">CSV</a> |
         <a href="/export_data?graph=comp&fmt=json" target="_blank">JSON</a></p>
    </div>
    """

# -------------- DEBUG LOGS --------------
@app.route("/debug_logs")
def debug_logs():
    lines= "<br>".join(LOG_BUFFER)
    return f"<h2>Debug Logs</h2><pre>{lines}</pre>"

# -------------- RUN FLASK --------------
def run_flask():
    log_msg(f"[DEBUG] run_flask => listening on port {PORT}")
    app.run(host="0.0.0.0", port=PORT, debug=False)

if __name__=="__main__":
    flask_thread= threading.Thread(target=run_flask, daemon=True)
    flask_thread.start()
    time.sleep(3)
    if SCENARIO=="domain":
        log_msg("🌐 SCENARIO=domain => check http://127.0.0.1:8000")
        while True:
            time.sleep(1)
    elif SCENARIO=="ngrok":
        log_msg("🌐 SCENARIO=ngrok => ephemeral link => default.")
        try:
            from pyngrok import ngrok
            ngrok.set_auth_token(NGROK_AUTH_TOKEN)
            public_url= ngrok.connect(PORT)
            log_msg(f"🌐 Public URL => {public_url.public_url}")
            log_msg("✅ Keep script running => Ctrl+C to stop.")
            while True:
                time.sleep(1)
        except Exception as e:
            log_msg(f"⚠️ Could not start ngrok => {e}")
            while True:
                time.sleep(1)
    elif SCENARIO=="cloudflared":
        log_msg("🌐 SCENARIO=cloudflared => ephemeral link.")
        while True:
            time.sleep(1)
    else:
        log_msg(f"❓ Unknown SCENARIO={SCENARIO}, exiting...")


KeyboardInterrupt: 